# 线性规划 Linear Programming
参考自《Python与运筹优化》

## 逻辑的数学表示
### 条件约束
$ B>0 \space if \space A>0$ 等价于
$$
s.t.
    \begin{cases}
    s \cdot A - max(A, 0) = 0 \\
    s \cdot M + B > 0 \\
    s = 0 \space or \space 1
    \end{cases}
$$ 
其中M为一个充分大的数

- 或逻辑：

$ B>0 \space if \space A_1>0 \space or \space A_2>0$ 等价于
$$
s.t.
    \begin{cases}
    s_1 \cdot A_1 - max(A, 0) = 0 \\
    s_2 \cdot A_2 - max(A, 0) = 0 \\
    (s_1 + s_2) \cdot M + B > 0 \\
    s_1, s_2 = 0 \space or \space 1
    \end{cases}
$$ 

- 与逻辑：

$ B>0 \space if \space A_1>0 \space and \space A_2>0$ 等价于
$$
s.t.
    \begin{cases}
    s_1 \cdot A_1 - max(A, 0) = 0 \\
    s_2 \cdot A_2 - max(A, 0) = 0 \\
    (|s_1 + s_2| - |s_1 - s_2|) \cdot M + B > 0 \\
    s_1, s_2 = 0 \space or \space 1
    \end{cases}
$$ 

## 运输问题

### 产销平衡且确定

In [13]:
import os
os.chdir('..')  # 改变工作路径

from rich import print
import numpy as np
from problem_solving.transportation_problem import balance_transportation_problem
##  示例
c = np.array([[3, 11, 3, 10],  # c[i, j] 为从A_i到B_j的单位运费
              [1, 9, 2, 8],
              [7, 4, 10, 5]])
sales_volume = np.array([3, 6, 5, 6]) # B_j销量 j = 0, 1, 2, 3
production = np.array([7, 4, 9])  # A_i产量

print(*balance_transportation_problem(c, production, sales_volume), sep='\n\n')

[[2. 0. 5. 0.]
 [1. 0. 0. 3.]
 [0. 6. 0. 3.]]

85.0

### 产销不平衡
- 修改线性规划代码
- 转换成平衡问题, 虚拟出一个垃圾倾销地, 或者一个新的产地

这里展示将问题转换成平衡问题

In [16]:
# 供过于求
production2 = production + 1
print(production2)

c2 = np.hstack((c, np.array([0, 0, 0]).reshape(3, 1)))
print(c2)

sales_volume2 = np.hstack((sales_volume, [np.sum(production2) - np.sum(sales_volume)]))
print(sales_volume2)

print(*balance_transportation_problem(c2, production2, sales_volume2), sep='\n\n')

[ 8  5 10]

[[ 3 11  3 10  0]
 [ 1  9  2  8  0]
 [ 7  4 10  5  0]]

[3 6 5 6 3]

[[0. 0. 5. 0. 3.]
 [3. 0. 0. 2. 0.]
 [0. 6. 0. 4. 0.]]

78.0

### 产销不确定

这里演示如何转换成平衡问题

In [35]:
Num = float | int

class RangeNum(object):
    def __init__(self, l) -> None:
        super().__init__()
        if isinstance(l, tuple):
            assert l[0] <= l[1]
            if l[0] == l[1]:
                self.lower_bound = self.upper_bound = l[0]
            else:
                self.lower_bound, self.upper_bound = l
            return
        assert isinstance(l, Num)
        self.lower_bound = self.upper_bound = l
    
    def is_no_range(self):
        return self.lower_bound == self.upper_bound
    
    def __str__(self) -> str:
        if self.is_no_range():
            return str(self.lower_bound)
        return f'{self.lower_bound}~{self.upper_bound}'

    def __repr__(self) -> str:
        return str(self)

sales_volume3 = np.array([RangeNum(i) for i in [3, (4, 7), 5, 6]])
print(sales_volume3)
M = 1000
c3 = c = np.array([[3, 11, 3, 10, 11],  # c[i, j] 为从A_i到B_j的单位运费
                   [1, 9, 2, 8, 9],
                   [7, 4, 10, 5, 4],
                   [M, M, M, M, 0]])
sales_volume3 = np.array([3, 4, 5, 6, 7-4])
production3 = np.array([7, 4, 9, 1])

print(*balance_transportation_problem(c3, production3, sales_volume3), sep='\n\n')

[3 4~7 5 6]

[[2. 0. 5. 0. 0.]
 [1. 0. 0. 3. 0.]
 [0. 4. 0. 3. 2.]
 [0. 0. 0. 0. 1.]]

85.0

这里的结果与第一例一样, 因为肯定不做无用功, 如果是有两个不确定, 这个例子就显得更有意义些